# Using Notebook Environments

In [ ]:
# Installing packages in Google Colab environment
# !pip install datasets transformers evaluate
# !pip install accelerate -U
#
# # Mounting google drive to enable access to data files
# from google.colab import drive
# drive.mount('/content/drive')
#
# # Changing working directory to ex1
# %cd /content/drive/MyDrive/LLM4JDM/ex1

# Preparing data
We begin by loading the requisite packages. For those coming from R, packages in Python are sometimes given shorter names for use in the code via the ```import <name> as <nickname>``` syntax (e.g. ```import pandas as pd```). These are usually standardized nicknames. We here make use of three packages:
1. ```pandas```: A very popular library for reading and manipulating data in python.
2. ```datasets```: A package from the HuggingFace (HF) ecosystem for loading and manipulating datasets in a format ready for use with HF models.
3. ```transformers```: A package from the HF ecosystem for loading and manipulating transfomer-based models.

In [43]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

The dataset takes the following structure:
1. ```text```:
2. ```labels```:

In [56]:
vaccine = pd.read_csv('vaccine.csv', dtype={'text': str, 'labels': float})
vaccine

,text,labels
0,Looking at the probability of certain negative...,0.625
1,"Not really, I looked at all the numbers and ma...",0.750
2,I weighed the side effects with the benefits a...,0.750
3,Percentages were important. In all things (med...,0.375
4,I would look up the potential side effects and...,0.250
...,...,...
1041,I looked primarily at the effectiveness of eac...,0.750
1042,Personal experience talking,0.125
1043,Biased against the vaccine,0.750
1044,I have had a lot of people whos gotten the vac...,0.250


In [58]:
# Converting pandas dataframe to HF Dataset
vaccine = Dataset.from_pandas(vaccine)
vaccine

Dataset({
    features: ['text', 'labels'],
    num_rows: 1046
})

Features of the ```Dataset``` object can be accessed like keys in a dictionary, and behave like python lists. Sample can be accessed by index, returning a dictionary where keys correspond to feature names. This reflects the fact that ```Datasets``` is based on Apache Arrow, which defines a typed columnar format that is more memory efficient than native Python

In [59]:
vaccine[0]

{'text': 'Looking at the probability of certain negative side effects and seeing if the vaccine genuinely protects someone would be very important.',
 'labels': 0.625}

To use models in the HF ecosystem, one must first define a model checkpoint (```ckpt```): the set of weights that are loaded into a given transformer architecture. This often needs to be done well before we even initialise the model, since data preprocessing steps such as tokenization must follow the steps used to train the original model. We just need a pretrained base model for our purposes (i.e. one that has not yet been fine-tuned on a specific task). One popular lightweight option is ```distilbert-base-uncased```.

In [60]:
# Defining model checkpoint
model_ckpt = 'distilbert-base-uncased'

Tokenization is the process of breaking raw text into the desired atomic units for one's modelling task. This may be as simple as splitting the text into individual characters. In the case of transformer-based models, tokenization is a bit more complex, usually occurring at the sub-word level.

In [61]:
# Tokenizing the dataset
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
print(f'Vocabulary size: {tokenizer.vocab_size}, max context length: {tokenizer.model_max_length}')

Vocabulary size: 30522, max context length: 512


Two important arguments relating to tokenization:
1. ```padding```: Used to fill up sequences to a certain length, ensuring that all sequences in a batch have the same length. This is essential for training and inference with deep learning models that operate on fixed-size input tensors.
2. ```truncation```: Truncation is the process of cutting off parts of the sequence to ensure that it fits within a specified maximum length (e.g. 512 tokens for BERT models)

The combination of padding and truncation ensures that all sequences in a batch conform to a consistent, fixed size, which is essential for processing in parallel on modern hardware like GPUs

In [62]:
#
tokenize = lambda batch: tokenizer(batch['text'], padding="max_length", truncation=True)

vaccine = vaccine.map(tokenize, batched=True)
vaccine[0]

Map:   0%|          | 0/1046 [00:00<?, ? examples/s]

{'text': 'Looking at the probability of certain negative side effects and seeing if the vaccine genuinely protects someone would be very important.',
 'labels': 0.625,
 'input_ids': [101,
  2559,
  2012,
  1996,
  9723,
  1997,
  3056,
  4997,
  2217,
  3896,
  1998,
  3773,
  2065,
  1996,
  17404,
  15958,
  18227,
  2619,
  2052,
  2022,
  2200,
  2590,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

Looking at the first sample, we see some important special tokens:
1. ```[CLS]```: Often added at the beginning of the input sequence. In the context of classification tasks, the embedding corresponding to the [CLS] token (after passing through the model) is often used as the aggregate representation for the entire sequence.
2. ```[SEP]```: Used to separate different segments in a sequence. For example, in tasks that take two different sentences as input (such as question-answering or text-pair classification), the [SEP] token is placed between the two sentences to indicate that they are distinct segments. This helps the model understand and process the two segments appropriately, recognizing the boundaries between them.

Make sure to scroll up to see the start of the sample.

In [ ]:
# Inspecting tokenization
tokenizer.convert_ids_to_tokens(vaccine[0]['input_ids'])

# Feature Extraction
We require two packages/modules for feature extraction:
1. ```torch```: The PyTorch package, which is the most popular deep learning framework amongst researchers (https://paperswithcode.com/trends).
2. ```Automodel```: A module from the HF ecosystem that allows us to load a pretrained model and use it for inference. This is a very convenient way to use pretrained models, since it abstracts away the details of the model architecture and allows us to focus on the task at hand.

In [ ]:
import torch
torch.manual_seed(42) # For reproducibility
from transformers import AutoModel

In order to pass our data to the model, we need to convert it to torch tensors. If you are familiar with NumPy, torch tensors are very similar, but with the added benefit of being able to run on GPUs (which are optimized for tensor operations).

In [ ]:
vaccine.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
vaccine

In [ ]:
# Loading the model and moving it to the GPU if available
if torch.cuda.is_available():  # for nvidia GPUs
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # for Apple Metal Performance Sharder (mps) GPUs
    device = torch.device('mps')
else:
    device = torch.device('cpu')

device

In [ ]:
# Loading distilbert-base-uncased and moving it to the GPU if available
model = AutoModel.from_pretrained(model_ckpt).to(device)
f'Model inputs: {tokenizer.model_input_names}'

In [ ]:
def extract_features(batch):
    # Each batch is a dictionary with keys corresponding to the feature names. We only need the input ids and attention masks
    inputs = {k:v.to(device) for k, v in batch.items() if k in tokenizer.model_input_names}

    # Tell torch not to build the computation graph during inference with `torch.no_grad()`
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state # Extract last hidden states

    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

vaccine = vaccine.map(extract_features, batched=True, batch_size=8)
vaccine['hidden_state'].shape

# Predicting vaccine decisions with features
To predict the decisions using the extracted features and evaluate prediction performance we will make use of ```sklearn``` (scikit-learn), a general machine learning library. Since we are dealing with high-dimensinoal embeddings, ordinary least squares regression runs a risk of overfitting. Instead, we will use a regularized (linear) regression model (```RidgeCV```). We evaluate model performance on a holdout test set using the coefficient of determination ($R^2$).

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split

In [ ]:
# Converting features to a pandas dataframe for compatibility with sklearn
embeds = pd.DataFrame(vaccine['hidden_state'])
embeds

In [ ]:
# Instantiating the RidgeCV model
regr = RidgeCV()
regr

In [ ]:
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embeds, vaccine['labels'], random_state=42)
f'Train size: {len(X_train)}, test size: {len(X_test)}'

In [ ]:
# Fitting the model and evaluating performance
regr.fit(X_train, y_train)
f'Test R2 = {regr.score(X_test, y_test).round(2)}'

*Exercise*: Feel free to try out different regression models (e.g. LassoCV): https://scikit-learn.org/stable/supervised_learning.html

# Pedicting vaccine decisions the LM fine-tuning
We here make use of three modules from the transformers library:
1. ```AutoModelForSequenceClassification```: Loads a pretrained model ready for fine-tuning it on sequence classification/regression labels.
2. ```TrainingArguments```: Specify training arguments such as the number of epochs, batch size, learning rate, etc.
3. ```Trainer```: Allows us to train a model using the training arguments and a training dataset.

We also employ the ```evaluate``` library to compute the coefficient of determination ($R^2$) on the test set.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

In [ ]:
# Splitting the data into train and test sets
vaccine = vaccine.train_test_split(test_size=0.2)
vaccine

The main difference here from the model we used for feature extraction is ```distilbert-base-uncased``` now has a classification/regression head attached. You will see a warning that some parts of the model are randomly initialized. This is normal since the head has not yet been trained.

In [ ]:
# Loading distilbert-base-uncased and moving it to the GPU if available
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=1) # num_labels=1 for regression
         .to(device))

model

In [ ]:
# Training the model
model_name = f"{model_ckpt}-finetuned-vaccine"
batch_size = 8


training_args = TrainingArguments(
    output_dir=model_name,  # output directory to save training checkpoints
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=1/batch_size, # log training metrics at every epoch
    evaluation_strategy="epoch", # evaluate at the end of every epoch
    num_train_epochs=7,
)


def compute_metrics(eval_preds):
    metric = evaluate.load("r_squared")
    preds, labels = eval_preds
    return {"r_squared": metric.compute(predictions=preds, references=labels)}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=vaccine['train'],
    eval_dataset=vaccine['test'],
    compute_metrics=compute_metrics,
)

trainer.train()

# Exercises